### Data processing

In [ ]:
import pandas as pd
import time
import re
# !pip install nltk
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import contractions

### Load data

In [ ]:
# Load dataframes
df_politics_Israel = pd.read_csv('data/politics_Israel_comments.csv')
df_politics_Palestine = pd.read_csv('data/politics_Palestine_comments.csv')
df_worldnews_Israel = pd.read_csv('data/worldnews_Israel_comments.csv')
df_worldnews_Palestine = pd.read_csv('data/worldnews_Palestine_comments.csv')
df_Israel_Israel = pd.read_csv('data/Israel_Israel_comments.csv')
df_Israel_Palestine = pd.read_csv('data/Israel_Palestine_comments.csv')
df_Palestine_Israel = pd.read_csv('data/Palestine_Israel_comments.csv')
df_Palestine_Palestine = pd.read_csv('data/Palestine_Palestine_comments.csv')

# Concatenate the DataFrames
df_combined = pd.concat([
    df_politics_Israel, df_politics_Palestine, 
    df_worldnews_Israel, df_worldnews_Palestine, 
    df_Israel_Israel, df_Israel_Palestine, 
    df_Palestine_Israel, df_Palestine_Palestine
])

# Print initial shape
print(f"Combined DataFrame shape before processing: {df_combined.shape}")

# Store original text in a new column for reference
df_combined['comment_original'] = df_combined['comment_text']

# Remove duplicate comments based on 'comment_text'
df_combined = df_combined.drop_duplicates(subset='comment_text')

# Remove comments with 1 or fewer upvotes
df_combined = df_combined[df_combined['upvotes'] > 1]

# Remove URLs
df_combined['comment_text'] = df_combined['comment_text'].apply(lambda x: re.sub(r'http\S+|www.\S+', '', x))

# Expand contractions
df_combined['comment_text'] = df_combined['comment_text'].apply(lambda x: contractions.fix(x))

# Lowercase conversion
df_combined['comment_text'] = df_combined['comment_text'].str.lower()

# Remove special characters except ".", "!","?", and ","
df_combined['comment_text'] = df_combined['comment_text'].apply(lambda x: re.sub(r'[^a-zA-Z0-9.,!?]', ' ', x))

# Renaming
df_combined.rename(columns={'comment_text': 'comment_processed'}, inplace=True)

# Remove row if processed text is below 4 words
df_combined = df_combined[df_combined['comment_processed'].apply(lambda x: len(x.split()) > 3)]

# Remove whitespaces and multiple spaces
df_combined['comment_processed'] = df_combined['comment_processed'].str.strip().str.replace(r'\s+', ' ', regex=True)

# New column without stopwords
ps = PorterStemmer()
stop_words = set(stopwords.words('english'))

df_combined['comment_no_stopwords'] = df_combined['comment_processed'].apply(
    lambda x: ' '.join([word for word in x.split() if word not in stop_words])
)

# Reorder columns
df_combined = df_combined[['created_date', 'subreddit_id', 'search_query', 'post_id', 'comment_id', 
                           'post_title', 'comment_original', 'comment_processed', 'comment_no_stopwords', 'upvotes', 
                           'post_url', 'comment_url']]

# Remove row if there is an empty field
df_combined = df_combined.dropna()

# Reset index
df_combined.reset_index(drop=True, inplace=True)

# Print shape after processing
print(f"Combined DataFrame shape after processing: {df_combined.shape}")

# Display final DataFrame
df_combined.head(2)

Example to compare original comment with processed comment

In [ ]:
# print out a comment URL and text
print(df_combined['comment_url'].iloc[0])
print(df_combined['comment_original'].iloc[0])
print(df_combined['comment_processed'].iloc[0])
print(df_combined['comment_no_stopwords'].iloc[0])

### Apply sentiment and Lix

In [ ]:
%run sentiment_analysis.ipynb
%run lix_calc_function.ipynb # Use this once we have made a Lix_Calc that should only contain the function

#Apply function "lix_calc3"
df_combined['lix_score'] = df_combined['comment_processed'].apply(lix_calc_used)

# Apply function "calculate_sentiment"
df_combined['sentiment_score'] = df_combined['comment_processed'].apply(calculate_sentiment_vader)

Plot the distributions
---


In [ ]:
dt = df_combined.copy()

fig, axes = plt.subplots(2, 3, figsize=(11, 11))
cols = ['upvotes', 'lix_score', 'sentiment_score']
df_clean = dt[cols]

axes[0, 1].set_title('Unscaled data', fontsize=16)
sns.scatterplot(ax = axes[0, 0], data=df_clean, x="upvotes", y="lix_score")
sns.scatterplot(ax = axes[0, 1], data=df_clean, x="upvotes", y="sentiment_score")
sns.scatterplot(ax = axes[0, 2], data=df_clean, x="sentiment_score", y="lix_score")

sns.histplot(ax = axes[1,0], data=dt, x="upvotes").set_title('Upvotes')
sns.histplot(ax = axes[1,1], data=dt, x="sentiment_score").set_title('Sentiment Score')
sns.histplot(ax = axes[1,2], data=dt, x="lix_score").set_title('Lix Score')

plt.tight_layout()
plt.show()


# Export dataframes

In [ ]:
# Choose which columns to save
columns_to_save = ['created_date', 'subreddit_id', 'search_query', 'post_id', 'comment_id', 
                   'post_title', 'comment_processed', 'comment_no_stopwords', 
                   'upvotes', 'lix_score', 'sentiment_score']

# Save the processed DataFrame to a CSV file
df_combined[columns_to_save].to_csv('data/combined_comments_processed.csv', index=False)

In [ ]:
# Now we export them to CSV
subreddits = df_combined['subreddit_id'].unique()
for subreddit in subreddits:
    df_combined[df_combined['subreddit_id'] == subreddit].to_csv(f'data/{subreddit}_w_lix_sentiment.csv', index=False)